In [1]:
## Install Dependencies


In [1]:
# # Install dependencies

! git clone https://github.com/deepanshu-yadav/Hindi_GramVani_Finetune.git
!pip install wget
!apt-get install sox libsndfile1 ffmpeg libsox-fmt-mp3
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

Cloning into 'Hindi_GramVani_Finetune'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 7), reused 15 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 47.37 KiB | 703.00 KiB/s, done.
Resolving deltas: 100% (7/7), done.
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=71775be4ea1433353cfe8890b26a881ea2b6aa1c60ad2b929680fe8a5bc76ab3
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0

In [ ]:
## Please re run the session after executing the cell above this.

In [ ]:
## Prepare the manifests files.

In [2]:
! cp Hindi_GramVani_Finetune/prepare_manifest.py .
! python prepare_manifest.py

100% [................................................] 2039605874 / 2039605874
Extracting train dataset...
Train dataset extracted to /kaggle/working/data/GigaVoice/GV_Train_100h
100% [....................................................] 98263244 / 98263244
Extracting val dataset...
Val dataset extracted to /kaggle/working/data/GigaVoice/GV_Dev_5h
100% [....................................................] 62063203 / 62063203
Extracting test dataset...
Test dataset extracted to /kaggle/working/data/GigaVoice/GV_Eval_3h
Resampling /kaggle/working/data/GigaVoice/GV_Train_100h/Audio: 100%|█| 37152/371
Resampling /kaggle/working/data/GigaVoice/GV_Dev_5h/Audio: 100%|█| 1885/1885 [00
Resampling /kaggle/working/data/GigaVoice/GV_Eval_3h/Audio: 100%|█| 1032/1032 [0
Processing /kaggle/working/data/GigaVoice/GV_Train_100h/text: 100%|█| 37152/3715
Created train_manifest.json with 37152 entries
Processing /kaggle/working/data/GigaVoice/GV_Dev_5h/text: 100%|█| 1885/1885 [00:
Created val_manifest.

In [ ]:
##  Tokenize the language

In [3]:
! cp Hindi_GramVani_Finetune/tokenize_language.py .
! python tokenize_language.py

Found 37152 texts for training
Saved raw text corpus to tokenizer_output/document.txt
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokenizer_output/document.txt
  input_format: 
  model_prefix: tokenizer_output/tokenizer
  model_type: BPE
  vocab_size: 1024
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <u

In [ ]:
## Download the checkpoint

In [4]:
! cp Hindi_GramVani_Finetune/store_parakeet.py .
! python store_parakeet.py


parakeet-tdt-0.6b-v2.nemo: 100%|████████████| 2.47G/2.47G [00:22<00:00, 109MB/s]
[NeMo I 2025-06-02 12:19:40 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo W 2025-06-02 12:19:41 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-06-02 12:19:41 nemo_logging:405] If you intend to do validation, please call t

In [ ]:
## Prepare the finetuning configuration (Already done for you)

In [5]:
! cp Hindi_GramVani_Finetune/finetune.py .
! cp Hindi_GramVani_Finetune/hindi_config.yaml .
! cat hindi_config.yaml


name: "Speech_To_Text_Finetuning_parakeet_v2_tdt_hindi"

# use `init_from_nemo_model` or `init_from_pretrained_model` to initialize the model
# We do not currently support `init_from_ptl_ckpt` to create a single script for all types of models.
init_from_nemo_model:  parakeet-tdt-0.6b-v2/parakeet-tdt-0.6b-v2.nemo  # path to nemo model

model:
  sample_rate: 16000

  train_ds:
    manifest_filepath: train_manifest.json
    sample_rate: ${model.sample_rate}
    batch_size: 4 # you may increase batch_size if your memory allows
    shuffle: true
    num_workers: 8
    pin_memory: true
    max_duration: 20
    min_duration: 0.1
    # tarred datasets
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    # bucketing params
    bucketing_strategy: "fully_randomized"
    bucketing_batch_size: null

  validation_ds:
    manifest_filepath: val_manifest.json
    sample_rate: ${model.sample_rate}
    batch_size: 4
    shuffle: false
    use_start_end_token: false
    num_work

In [ ]:
## We can fine now

In [ ]:
! python finetune.py

[NeMo I 2025-06-02 12:20:29 nemo_logging:393] Hydra config: name: Speech_To_Text_Finetuning_parakeet_v2_tdt_hindi
    init_from_nemo_model: parakeet-tdt-0.6b-v2/parakeet-tdt-0.6b-v2.nemo
    model:
      sample_rate: 16000
      train_ds:
        manifest_filepath: train_manifest.json
        sample_rate: ${model.sample_rate}
        batch_size: 4
        shuffle: true
        num_workers: 8
        pin_memory: true
        max_duration: 20
        min_duration: 0.1
        is_tarred: false
        tarred_audio_filepaths: null
        shuffle_n: 2048
        bucketing_strategy: fully_randomized
        bucketing_batch_size: null
      validation_ds:
        manifest_filepath: val_manifest.json
        sample_rate: ${model.sample_rate}
        batch_size: 4
        shuffle: false
        use_start_end_token: false
        num_workers: 8
        pin_memory: true
      test_ds:
        manifest_filepath: test_manifest.json
        sample_rate: ${model.sample_rate}
        batch_size: 4
  